# LVM Guide Star Catalog using the new library

In [ ]:
import lvmguiding
import numpy as np
import os
import matplotlib.pyplot as plt
import time
from astropy.coordinates import SkyCoord
from importlib import reload
from matplotlib.colors import LogNorm

## Creating the pointing list

In [ ]:
!pwd

In [ ]:
lvmguiding.lvminst.mag_lim_lower = 99

In [ ]:
if False: #Old way of creating the pointing list (a sparse grid for testing purposes):
    pointing_list = []

    counter = 0
    ra_list =[]
    dec_list=[]
    for ra in np.arange(5,355,10):
        for dec in np.arange(-85,85,10):
            c = SkyCoord(frame='icrs',ra=ra, dec=dec,unit='deg')
            pointing_list.append(c)
            ra_list.append(ra)
            dec_list.append(dec)
    #pointing_list=pointing_list[:10]

    indices = np.array(range(len(pointing_list)))

    output = np.stack((indices,ra_list,dec_list),axis=1)

    os.system("mkdir /data/beegfs/astro-storage/groups/others/neumayer/haeberle/lvm_outsourced/guide_star_search_results_no_faint_limit")
    filename = "/data/beegfs/astro-storage/groups/others/neumayer/haeberle/lvm_outsourced/guide_star_search_results_no_faint_limit/pointing_list"

    #pointing_list = pointing_list[[1,4,34,56,70,100]]
    if True: np.savetxt(filename,output,fmt="%10.6f")

    #pointing_list = pointing_list[::50]

else: #new way of creating the pointing list usin the results from the survey simulator
    from astropy.io import fits
    with fits.open("lvmsurveysim_hz_1000.fits") as hdul:
        my_data = hdul[1].data

    my_data = my_data[my_data["ra"]>-500]

    ra_list = my_data["ra"]
    dec_list = my_data["dec"]

    indices = np.array(range(len(ra_list)))




    pointing_list = []

    for index in indices:
        ra = ra_list[index]
        dec = dec_list[index]

        c = SkyCoord(frame='icrs',ra=ra, dec=dec,unit='deg')
        pointing_list.append(c)

    output = np.stack((indices,ra_list,dec_list),axis=1)


    filename = "/data/beegfs/astro-storage/groups/others/neumayer/haeberle/lvm_outsourced/guide_star_search_results_no_faint_limit/pointing_list"

    #pointing_list = pointing_list[[1,4,34,56,70,100]]
    if True:
        np.savetxt(filename,output,fmt="%10.6f")

fig,ax = plt.subplots(figsize=(12,6))
ax.plot(ra_list,dec_list,"k.",ms=1)
ax.set_xlabel("RA")
ax.set_ylabel("DEC") 
ax.invert_xaxis()

    
pointing_touple_list = []
for index,c in enumerate(pointing_list):
    pointing_touple_list.append((index,c))

## Paralellized work on the pointing list

In [ ]:
if True: 
    
    from multiprocessing import Pool

    t00 = time.time()
    num_processors = 28#Create a pool of processors
    p=Pool(processes = num_processors)#get them to work in parallel
    output = p.map(lvmguiding.find_guide_stars_auto,[input_touple for input_touple in pointing_touple_list[:]])
    p.close()
    p.join()

    for o in output:
        print(o)

    t11 = time.time()

    print("Total execution time: {:.1f} s".format(t11-t00))

In [ ]:
lvmguiding.lvminst.mag_lim_lower

## Making some example plots

In [ ]:
#What is going on at <SkyCoord (ICRS): (ra, dec) in deg
#    (80.01870621, -68.35233157)>

In [ ]:
reload(lvmguiding)

In [ ]:

c = pointing_touple_list[50][1]
print(c)
color_array=["r","g","b","r","g","b"]


fig,ax = plt.subplots(figsize=(12,12))

for index,pa in enumerate([0,60,120,180,240,300]):
        print("PA: ",pa)
        if pa==0:
            print("Using the full cat for the first position")
            #culled_cat=lvmguiding.cat_full
            culled_cat = lvmguiding.get_cat_using_healpix2(c,plotflag=True)
        ras,decs,dd_x_mm,dd_y_mm,chip_xxs,chip_yys,mags,culled_cat = lvmguiding.find_guide_stars(c,pa=pa,plotflag=False,recycled_cat=culled_cat)
        
        if pa==0:
            ax.plot(culled_cat["ra"],culled_cat["dec"],"k.",ms=1)
        ax.plot(ras,decs,".",c=color_array[index],label="PA = "+str(pa)+" deg")
        
ax.legend()

ax.set_xlabel("RA [deg]")
ax.set_ylabel("DEC [deg]")
ax.invert_xaxis()

#fig.savefig("/home/haeberle/exchange/lvm/report/position_angles.png",bbox_inches="tight",facecolor="w",edgecolor="w",dpi=200)

In [ ]:
reload(lvmguiding)

In [ ]:
c = pointing_touple_list[5419][1]
c = pointing_touple_list[3429][1]
#c = pointing_touple_list[1017][1]
print(c)
color_array=["r","r","r"]

my_inst = lvmguiding.InstrumentParameters()

my_inst.mag_lim_lower=17

fig,ax = plt.subplots(figsize=(6,6))

for index,pa in enumerate([0,180]):
        print("PA: ",pa)
        if pa==0:
            print("Using the full cat for the first position")
            #culled_cat=lvmguiding.cat_full
            culled_cat = lvmguiding.get_cat_using_healpix2(c,plotflag=True)
        ras,decs,dd_x_mm,dd_y_mm,chip_xxs,chip_yys,mags,culled_cat = lvmguiding.find_guide_stars(c,pa=pa,plotflag=False,recycled_cat=culled_cat,inst=my_inst)
        
        if pa==0:
            ax.plot(culled_cat["ra"],culled_cat["dec"],"ko",ms=1)

        ax.plot(ras,decs,"o",c=color_array[index],label="PA = "+str(pa)+" deg",ms=1)
        
ax.legend()

ax.set_xlabel("RA [deg]")
ax.set_ylabel("DEC [deg]")
ax.invert_xaxis()

fig.savefig("../../../../exchange_temp/median_density.png",dpi=200,facecolor="w",edgecolor="w",bbox_inches="tight")

## Create a synthetic image

In [ ]:
culled_cat = lvmguiding.get_cat_using_healpix2(c,plotflag=True)
ras,decs,dd_x_mm,dd_y_mm,chip_xxs,chip_yys,mags,culled_cat = lvmguiding.find_guide_stars(c,pa=0,plotflag=False,recycled_cat=culled_cat)

In [ ]:
standard_instrument = lvmguiding.InstrumentParameters()

In [ ]:
my_image = lvmguiding.make_synthetic_image(chip_x=chip_xxs,
                                           chip_y=chip_yys,
                                          gmag=mags,
                                          inst =standard_instrument,
                                          exp_time=5,
                                          seeing_arcsec=3.5,
                                          sky_flux=15)

In [ ]:
combined= my_image
fig,ax4 = plt.subplots(figsize=(12,8))

vmin4 = np.percentile(combined,25)
vmax4 = np.percentile(combined,99.5)

my_plot4 = ax4.imshow(combined,origin="lower",norm=LogNorm(vmin=np.max([vmin4,1]), vmax=vmax4))


#if np.sum(sn>5) < 10:
#    ax4.set_title("Combined (Bias + Readout Error + Noisy Background + Noisy Stars)\nBrightest star (red): gmag = {:.2f} ; F = {:.1f} e-/s\nPotentially saturated pixels (Ne- > 20000): {}\nWhite circle around all stars with S/N > 5 (N = {})".format(np.min(gmag),gaia_flux[np.argmin(gmag)],np.sum(combined>20000),np.sum(sn>5)))
#    ax4.plot(x_position[sn>5],y_position[sn>5],"o",ms=40,markerfacecolor="none",markeredgecolor="w",label="gmag < 12")
#else:
#    ax4.set_title("Combined (Bias + Readout Error + Noisy Background + Noisy Stars)\nPointing: {}  Texp: {} s\n{} of {} stars have a S/N > 5\nBrightest star (red): gmag = {:.2f} ; F = {:.1f} e-/s\nPotentially saturated pixels (Ne- > 20000): {}\n".format(pointing_string,exp_time,np.sum(sn>5),len(gmag),np.min(gmag),gaia_flux[np.argmin(gmag)],np.sum(combined>20000)))
#ax4.plot(x_position[np.argmin(gmag)],y_position[np.argmin(gmag)],"o",ms=20,markerfacecolor="none",markeredgecolor="r",label="gmag < 12")
plt.colorbar(my_plot4,ax=ax4,fraction=0.046, pad=0.04)
#plt.colorbar(my_plot4,ax=ax,fraction=0.046, pad=0.04)
#ax4.legend()

fig.tight_layout()

#filename = "/home/haeberle/exchange/lvm/report/pointing_"+pointing_string+"_{:d}ms.png".format(int(1000*exp_time))
#fig.suptitle()
#fig.savefig("/home/haeberle/exchange/lvm/report/example_crowded_field.png",dpi=200,bbox_inches="tight",edgecolor="white",facecolor="white")
#fig.savefig(filename,dpi=200,bbox_inches="tight",edgecolor="white",facecolor="white") 